In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

import numpy as np 
from class_functions import *
import pandas as pd
import sklearn

np.random.seed(1234)
root_dir = "/home/alberto_sinigaglia/gaia"

In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
def basic_block(x, kernel_size, filters, name, pool_size=2):
    """Single Conv1D layer with ReLU activation + global avarage pooling"""

    x = tf.keras.layers.Conv1D(filters, kernel_size, padding='same', activation=None, kernel_regularizer=tf.keras.regularizers.l2(1e-4), name=f'{name}_conv')(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.1, name=f"{name}_leaky_relu")(x)
    x = tf.keras.layers.AveragePooling1D(pool_size,  name=f'{name}_avgpooling')(x)
    return x
    
def residual_block(x, kernel_size, filters, name):

    shortcut = x 
    x = tf.keras.layers.Conv1D(filters, kernel_size, padding='same', activation=None, kernel_regularizer=tf.keras.regularizers.l2(1e-4), name=f'{name}_conv1')(x)
    x = tf.keras.layers.BatchNormalization(name=f'{name}_bn1')(x)
    x = tf.keras.layers.LeakyReLU(negative_slope=0.1, name=f"{name}_leaky_relu1")(x)
    x = tf.keras.layers.Conv1D(filters, kernel_size, padding='same', activation=None,  kernel_regularizer=tf.keras.regularizers.l2(1e-4), name=f'{name}_conv2')(x)
    x = tf.keras.layers.BatchNormalization(name=f'{name}_bn2')(x)
    x = tf.keras.layers.Add( name=f'{name}_add')([shortcut, x])
    x = tf.keras.layers.LeakyReLU(negative_slope=0.1, name=f"{name}_leaky_relu2")(x)
    return x

def build_model(input_length=6144, channels=1, filters=42):

    inp = tf.keras.layers.Input(shape=(input_length, channels), name='input_layer')

    x = basic_block(inp, 16, filters, name='bb_1')
    x = residual_block(x, 16, filters, name='rb_1')

    x = basic_block(x, 32, filters, name='bb_2')
    x = residual_block(x, 32, filters, name='rb_2')

    x = basic_block(x, 64, filters, name='bb_3')
    x =  residual_block(x, 64, filters, name='rb_3')

    x = basic_block(x, 64, filters, name='bb_4')

    x = tf.keras.layers.Flatten(name='flatten_layer')(x)

    x = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4), name='dl_1')(x)
    x = tf.keras.layers.Dropout(0.2, name='dropout_1')(x)
    x = tf.keras.layers.Dense(64,  activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4), name='dl_2')(x)
    x = tf.keras.layers.Dropout(0.2, name='dropout_2')(x)
    x = tf.keras.layers.Dense(64,  activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4), name='dl_3')(x)
    x = tf.keras.layers.Dropout(0.2, name='dropout_3')(x)
    x = tf.keras.layers.Dense(32,  activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4), name='dl_4')(x)

    out = tf.keras.layers.Dense(2, activation='linear', name='output_layer')(x)

    model =  tf.keras.Model(inp, out, name='regression_model')
    return model

model = build_model()
model.summary()